In [2]:
import pandas as pd
from datetime import datetime, timedelta

# Load your datasets into pandas DataFrames
customers = pd.read_csv('customers.csv')
products = pd.read_csv('products.csv')
ratings = pd.read_csv('ratings.csv')
orders = pd.read_csv('orders.csv')

# Ensure 'order_date' is in datetime format
orders['order_date'] = pd.to_datetime(orders['order_date'])

# Calculate Revenue (price * quantity) for each order
orders = orders.merge(products[['product_id', 'price']], on='product_id', how='left')
orders['revenue'] = orders['price'] * orders['quantity']

# Top-performing products by revenue
revenue_per_product = orders.groupby('product_id').agg(
    total_revenue=('revenue', 'sum'),
    total_units_sold=('quantity', 'sum')
).reset_index()

# Merge with product details to get product names
revenue_per_product = revenue_per_product.merge(products[['product_id', 'product_name']], on='product_id', how='left')

# Sort by total revenue and units sold
top_products_by_revenue = revenue_per_product.sort_values(by='total_revenue', ascending=False)
top_products_by_units_sold = revenue_per_product.sort_values(by='total_units_sold', ascending=False)

# Get the last month
last_month_start = (datetime.now() - timedelta(days=30)).date()

# Filter orders for the last month
orders_last_month = orders[orders['order_date'].dt.date >= last_month_start]

# Top clients for the last month (by revenue)
top_clients_by_revenue = orders_last_month.groupby('customer_id').agg(
    total_revenue=('revenue', 'sum')
).reset_index()

# Merge with customer details to get customer names
top_clients_by_revenue = top_clients_by_revenue.merge(customers[['customer_id', 'name']], on='customer_id', how='left')

# Sort by total revenue
top_clients_by_revenue = top_clients_by_revenue.sort_values(by='total_revenue', ascending=False)

# Top clients for the last month (by units sold)
top_clients_by_units_sold = orders_last_month.groupby('customer_id').agg(
    total_units_sold=('quantity', 'sum')
).reset_index()

# Merge with customer details to get customer names
top_clients_by_units_sold = top_clients_by_units_sold.merge(customers[['customer_id', 'name']], on='customer_id', how='left')

# Sort by total units sold
top_clients_by_units_sold = top_clients_by_units_sold.sort_values(by='total_units_sold', ascending=False)

# Display the results
print("Top Products by Revenue:")
print(top_products_by_revenue[['product_name', 'total_revenue', 'total_units_sold']])

print("\nTop Products by Units Sold:")
print(top_products_by_units_sold[['product_name', 'total_revenue', 'total_units_sold']])

print("\nTop Clients by Revenue in the Last Month:")
print(top_clients_by_revenue[['name', 'total_revenue']])

print("\nTop Clients by Units Sold in the Last Month:")
print(top_clients_by_units_sold[['name', 'total_units_sold']])

Top Products by Revenue:
               product_name  total_revenue  total_units_sold
5       Brimnes Bed Storage          59521                77
41         Småstad Wardrobe          51688                52
28             Råskog Stool          50445                59
25            Docksta Table          46386                54
30             Nockeby Sofa          45646                58
4             Hemnes Daybed          44785                53
33       Fjälla Storage Box          43384                44
47           Bestå TV Bench          42320                46
0               Ektorp Sofa          40946                59
45       Valje Wall Cabinet          40140                60
36              Fredde Desk          39422                46
26             Ivar Cabinet          37785                55
9      Kallax Shelving Unit          37383                51
43          Koppang Dresser          36720                48
38    Mackapar Shoe Storage          36640                80

In [10]:
import pandas as pd
from datetime import datetime

# Load your datasets
customers = pd.read_csv('customers.csv')
orders = pd.read_csv('orders.csv')

# Ensure the order_date is in datetime format
orders['order_date'] = pd.to_datetime(orders['order_date'])

# Get the current date for recency calculation
current_date = datetime.now()

# Calculate Recency (days since last purchase)
recency = orders.groupby('customer_id')['order_date'].max().reset_index()
recency['Recency'] = (current_date - recency['order_date']).dt.days

# Calculate Frequency (number of orders per customer)
frequency = orders.groupby('customer_id').size().reset_index(name='Frequency')

# Calculate Monetary (total spending per customer)
orders = orders.merge(products[['product_id', 'price']], on='product_id', how='left')
orders['revenue'] = orders['price'] * orders['quantity']
monetary = orders.groupby('customer_id')['revenue'].sum().reset_index(name='Monetary')

# Merge Recency, Frequency, and Monetary data
rfm = recency[['customer_id', 'Recency']].merge(frequency[['customer_id', 'Frequency']], on='customer_id', how='left')
rfm = rfm.merge(monetary[['customer_id', 'Monetary']], on='customer_id', how='left')

# Calculate RFM score (optional, for segmentation)
# You can create a score for each segment by assigning quantiles (e.g., 1 = low, 5 = high) for each RFM metric

# For simplicity, let's assign quantiles for each metric:
rfm['Recency_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm['Frequency_Score'] = pd.qcut(rfm['Frequency'], 5, labels=[1, 2, 3, 4, 5])
rfm['Monetary_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

# Calculate the total RFM score (optional, can be used for segmentation)
rfm['RFM_Score'] = rfm['Recency_Score'].astype(int) + rfm['Frequency_Score'].astype(int) + rfm['Monetary_Score'].astype(int)

# Segment customers based on RFM score
def rfm_segment(x):
    if x >= 12:
        return 'Best Customers'
    elif x >= 9:
        return 'Loyal Customers'
    elif x >= 6:
        return 'Potential Loyalists'
    elif x >= 3:
        return 'Recent Customers'
    else:
        return 'At Risk'

rfm['Segment'] = rfm['RFM_Score'].apply(rfm_segment)



rfm.head()  # Displaying first few rows of the RFM table for inspection

,customer_id,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,RFM_Score,Segment
0,1,493,18,27081,5,5,5,15,Best Customers
1,2,495,9,9347,4,2,2,8,Potential Loyalists
2,3,497,13,18077,3,4,5,12,Best Customers
3,4,496,22,25717,4,5,5,14,Best Customers
4,5,501,11,14913,2,3,4,9,Loyal Customers


In [7]:
import pandas as pd

# Load the ratings dataset
ratings = pd.read_csv('ratings.csv')
products = pd.read_csv('products.csv')

# Top-reviewed products (i.e., most reviews)
top_reviewed_products = ratings.groupby('product_id').size().reset_index(name='num_reviews')

# Calculate average rating for each product
average_rating_per_product = ratings.groupby('product_id')['rating'].mean().reset_index(name='average_rating')

# Merge the number of reviews and average ratings
top_reviewed_products = top_reviewed_products.merge(average_rating_per_product, on='product_id', how='left')

# Merge with the product names
top_reviewed_products = top_reviewed_products.merge(products[['product_id', 'product_name']], on='product_id', how='left')

# Sort by number of reviews in descending order to get the top-reviewed products
top_reviewed_products_sorted = top_reviewed_products.sort_values(by='num_reviews', ascending=False)

# Display top 10 reviewed products with average rating
print("Top Reviewed Products with Average Ratings:")
print(top_reviewed_products_sorted[['product_name', 'num_reviews', 'average_rating']].head(10))

# Average number of reviews per customer
reviews_per_customer = ratings.groupby('customer_id').size().reset_index(name='num_reviews_per_customer')

# Calculate the average number of reviews per customer
average_reviews_per_customer = reviews_per_customer['num_reviews_per_customer'].mean()

print(f"\nAverage Number of Reviews per Customer: {average_reviews_per_customer}")

Top Reviewed Products with Average Ratings:
               product_name  num_reviews  average_rating
11           Raskog Trolley           16        2.312500
37  Bekant Conference Table           14        3.428571
6           Lack Side Table           13        3.000000
30             Nockeby Sofa           12        3.000000
44         Hektar Work Lamp           11        3.454545
31             Kivik Chaise           11        3.181818
27         Tarva Nightstand           11        2.545455
26             Ivar Cabinet           11        3.000000
16   Sinnerlig Pendant Lamp           11        2.909091
29     Strandmon Wing Chair           11        2.909091

Average Number of Reviews per Customer: 4.040404040404041


In [11]:
import pandas as pd

# Load your datasets
customers = pd.read_csv('customers.csv')
orders = pd.read_csv('orders.csv')
ratings = pd.read_csv('ratings.csv')
products = pd.read_csv('products.csv')

# Calculate the total amount spent by each customer
# Merge orders with products to get the price per product
orders = orders.merge(products[['product_id', 'price']], on='product_id', how='left')
orders['revenue'] = orders['price'] * orders['quantity']

# Calculate the total revenue per customer
total_spent = orders.groupby('customer_id')['revenue'].sum().reset_index(name='total_spent')

# Calculate the last purchase date for each customer
last_purchase = orders.groupby('customer_id')['order_date'].max().reset_index(name='last_purchase_date')

# Calculate the number of reviews left by each customer
reviews_per_customer = ratings.groupby('customer_id').size().reset_index(name='num_reviews')

# Merge all the calculated information into the customers DataFrame
customers = customers.merge(total_spent, on='customer_id', how='left')
customers = customers.merge(last_purchase, on='customer_id', how='left')
customers = customers.merge(reviews_per_customer, on='customer_id', how='left')

# Fill any NaN values (e.g., for customers who haven't spent money or left reviews)
customers['total_spent'] = customers['total_spent'].fillna(0)
customers['last_purchase_date'] = customers['last_purchase_date'].fillna('Never')
customers['num_reviews'] = customers['num_reviews'].fillna(0)



# Optionally, display the first few rows of the updated DataFrame in the console
customers.head()

,customer_id,name,total_spent,last_purchase_date,num_reviews
0,1,Customer_1,27081,2023-09-17 08:39:23.971834,4.0
1,2,Customer_2,9347,2023-09-15 08:39:23.971834,2.0
2,3,Customer_3,18077,2023-09-13 08:39:23.971834,4.0
3,4,Customer_4,25717,2023-09-14 08:39:23.971834,7.0
4,5,Customer_5,14913,2023-09-09 08:39:23.971834,2.0


In [12]:
y = 2+3*5.

In [13]:
y

17.0